In [45]:
# part 2
import tensorflow as tf
# Model parameters
def trainWithLearningRate(learning_rate):
    W = tf.Variable([.3], dtype=tf.float32)
    b = tf.Variable([-.3], dtype=tf.float32)
    # Model input and output
    x = tf.placeholder(tf.float32)
    linear_model = W*x + b
    y = tf.placeholder(tf.float32)
    # loss
    loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    # training data
    x_train = [1, 2, 3, 4]
    y_train = [0, -1, -2, -3]
    # training loop
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init) # reset values to wrong
    for i in range(1000):
      sess.run(train, {x: x_train, y: y_train})
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    return {"learningRate": learning_rate,"loss": curr_loss}

learning_rates = [0.01,0.001,0.1,0.5]

result = []
for learning_rate in learning_rates:
    result.append(trainWithLearningRate(learning_rate))

print("All the result")
print(result)

# The best learning rate is 0.01

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11
W: [-0.8397583] b: [0.52886957] loss: 0.14828278
W: [nan] b: [nan] loss: nan
W: [nan] b: [nan] loss: nan
All the result
[{'learningRate': 0.01, 'loss': 5.6999738e-11}, {'learningRate': 0.001, 'loss': 0.14828278}, {'learningRate': 0.1, 'loss': nan}, {'learningRate': 0.5, 'loss': nan}]


In [196]:
# part 3
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def mnistWithLearningRate(mnist,learning_rate):
  # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    # The raw formulation of cross-entropy,
    #
    #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
    #                                 reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
    # outputs of 'y', and then average across the batch.
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # Train
    for _ in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels})
    print(accuracy)
    return {"learning_rate": learning_rate, "Accuracy": accuracy}
    

def main(_):
  # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    learning_rates = [0.01,0.1,0.3,0.5,0.7]
    result = []
    for rate in learning_rates:
        result.append(mnistWithLearningRate(mnist,rate))
    print(result)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
0.8714
0.91
0.9187
0.9179
0.9167
[{'learning_rate': 0.01, 'Accuracy': 0.8714}, {'learning_rate': 0.1, 'Accuracy': 0.91}, {'learning_rate': 0.3, 'Accuracy': 0.9187}, {'learning_rate': 0.5, 'Accuracy': 0.9179}, {'learning_rate': 0.7, 'Accuracy': 0.9167}]


SystemExit: 

/Users/lifengxiang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [198]:
# part 4
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import numpy

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def mnistWithLearningRate(N,mode):
  # Create the model
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    x = tf.placeholder(tf.float32, [None, 784])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    y = tf.matmul(x, W) + b

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    # The raw formulation of cross-entropy,
    #
    #   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
    #                                 reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
    # outputs of 'y', and then average across the batch.
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    # Train

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        if (i + 1) % N == 0:
            if mode == "Sequential":
                numpy.random.shuffle(batch_ys)
            elif mode == "Random":
                m,n = batch_ys.shape
                for i in range(m):
                    randomCol = np.random.randint(10)
                    while batch_ys[i][randomCol] == 1:
                        randomCol = np.random.randint(10)
                    batch_ys[i] = np.zeros(10)
                    batch_ys[i][randomCol] = 1

            elif mode == "Maximum":
                m,n = batch_ys.shape
                for i in range(m):
                    if batch_ys[i][0] == 1:
                        batch_ys[i][0] = 0
                        batch_ys[i][6] = 1
                    elif batch_ys[i][1] == 1:
                        batch_ys[i][1] = 0
                        batch_ys[i][7] = 1
                    elif batch_ys[i][2] == 1:
                        batch_ys[i][2] = 0
                        batch_ys[i][5] = 1
                    elif batch_ys[i][3] == 1:
                        batch_ys[i][3] = 0
                        batch_ys[i][8] = 1
                    elif batch_ys[i][4] == 1:
                        batch_ys[i][4] = 0
                        batch_ys[i][9] = 1
        
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    accuarcy = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels})
    print("\n")
    print("batch size is " + str(N) , "mode is " + mode)
    accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels})
    print(accuracy)
    return {"N": N,"Mode": mode,"Accuracy": accuracy}
    
def main(_):
  # Import data
    
    batch_sizes = [5,10,50]
    modes = ["Sequential","Random","Maximum"]

    result = []
    for mode in modes:
        for batch_size in batch_sizes:
            result.append(mnistWithLearningRate(batch_size,mode))
    print(result)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str, default='/tmp/tensorflow/mnist/input_data',
                      help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


batch size is 5 mode is Sequential
0.8549
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


batch size is 10 mode is Sequential
0.854
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


batch size is 50 mode is Sequential
0.9135
Extracting /tmp/tenso

SystemExit: 

/Users/lifengxiang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
